In [1]:
from binance import Client
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from threading import Thread
client = Client()

In [5]:
symbols = ["ETHUSDT", "BTCUSDT", "ADAUSDT", "SOLUSDT", "ATOMUSDT", "MATICUSDT", "SHIBUSDT", "LTCUSDT", "TRXUSDT", "MANAUSDT", "DOTUSDT", "ALGOUSDT"]

In [6]:
def tf(symbol):
    klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1MINUTE, "500 day ago UTC")
    open_prices = [float(x[1]) for x in klines]
    high_prices = [float(x[2]) for x in klines]
    low_prices = [float(x[3]) for x in klines]
    close_prices = [float(x[4]) for x in klines]
    volume = [float(x[5]) for x in klines]
    df = pd.DataFrame(data = {"OPEN": open_prices, "HIGH": high_prices, "LOW": low_prices,"CLOSE":close_prices, "VOLUME":volume})
    df.to_csv("df/"+symbol+".csv")

    
threads = []
for i in tqdm(symbols):
    t = Thread(target = tf, args = (i,))
    t.start()
    threads.append(t)
    
for i in tqdm(range(len(threads))):
    threads[i].join()

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [7]:
import pickle
import ta
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import os

class candle_class:
    pass

def create_candles(df):
        sma21_raw = list(ta.trend.SMAIndicator(df["CLOSE"], 21).sma_indicator())
        #sma50_raw = list(ta.trend.SMAIndicator(df["CLOSE"], 50).sma_indicator())
        sma200_raw = list(ta.trend.SMAIndicator(df["CLOSE"], 200).sma_indicator())
        rsi_14 = [x / 50 - 1 for x in list(ta.momentum.RSIIndicator(df["CLOSE"], 14).rsi())]
        #atr_value = list(ta.volatility.AverageTrueRange(df["HIGH"], df["LOW"], df["CLOSE"], 200).average_true_range())
        #atr_fast = list(ta.volatility.AverageTrueRange(df["HIGH"], df["LOW"], df["CLOSE"], 9).average_true_range())
        #donchi = ta.volatility.DonchianChannel(df["HIGH"], df["LOW"], df["CLOSE"], 21)
        #dh = list(donchi.donchian_channel_hband())
        #dl = list(donchi.donchian_channel_lband())

        candles = []
        for i in range(len(df)):
          new_candle = candle_class()
          #new_candle.t = i
          #new_candle.o = df.iloc[i]["OPEN"]
          new_candle.h = df.iloc[i]["HIGH"]
          new_candle.l = df.iloc[i]["LOW"]
          new_candle.c = df.iloc[i]["CLOSE"]
          new_candle.v = df.iloc[i]["VOLUME"]


          #new_candle.dh = dh[i]
          #new_candle.dl = dl[i]
          new_candle.sma21 = sma21_raw[i]
          #new_candle.sma50 = sma50_raw[i]
          new_candle.sma200 = sma200_raw[i]
          new_candle.rsi14 = rsi_14[i]
          #new_candle.atr_value = atr_value[i]
          #new_candle.atr_fast = atr_fast[i]

          candles.append(new_candle)
        
        return candles
    

def Save(ob, file):
    f = open(file, "wb")
    pickle.dump(ob, f)
    f.close()
    
    
files = os.listdir("df")
for i in tqdm(files):
        if ".csv" in i:
            df = pd.read_csv("df/"+i) 
            df.columns = ["T", "OPEN", "HIGH", "LOW", "CLOSE", "VOLUME"]
            
            candles = create_candles(df)
            Save(candles, "./candles/candles_object_"+str(i))    

  0%|          | 0/12 [00:00<?, ?it/s]

In [8]:
%cd candles
import os
import zipfile
files = os.listdir(".")

for f in files:
    file = f+".zip"  # zip file name
    with zipfile.ZipFile(file, 'w',compression=zipfile.ZIP_DEFLATED) as zip:
        zip.write(f)
        
%cd ..

C:\Users\Root\Documents\jupy\cryptotest\candles
C:\Users\Root\Documents\jupy\cryptotest
